# DATA PRE-PROCESSING

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv("LT.csv")
x = dataset.iloc[1:, 1:5].values
y = dataset.iloc[1:, -1].values

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(x[:, :-1])
x[:, :-1] = imputer.transform(x[:, :-1])

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2)

# LINEAR REGRESSION MULTIVARIABLE


In [ ]:
from sklearn.linear_model import LinearRegression
lr_m =  LinearRegression()
lr_m.fit(xtrain,ytrain)
lr_m_pred = lr_m.predict(xtest)

In [ ]:
np.set_printoptions(precision=2)
print(np.concatenate((ytest.reshape(len(ytest),1), lr_m.predict(xtest).reshape(len(lr_m.predict(xtest)),1)),1))

[[1399.05 1400.87]
 [2040.55 2028.77]
 [1357.45 1350.06]
 ...
 [1392.6  1418.73]
 [1199.4  1204.2 ]
 [1628.75 1658.46]]


# LINEAR REGRESSION SINGLE VARIABLE

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain[:,0].reshape(-1,1), ytrain)

LinearRegression()

In [ ]:
lr_pred = lr.predict(xtest[:,0].reshape(-1,1))
print(np.concatenate((ytest.reshape(-1,1), lr_pred.reshape(-1,1)),-1))

[[1399.05 1399.27]
 [2040.55 2032.47]
 [1357.45 1353.73]
 ...
 [1392.6  1437.09]
 [1199.4  1207.61]
 [1628.75 1572.76]]


# POLYNOMIAL LINEAR REGRESSION

# SUPPORT VECTOR REGRESSION

# DECISION TREE REGRESSION

# RANDOM FOREST REGRESSION

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rand_reg = RandomForestRegressor(n_estimators=10)
rand_reg.fit(xtrain, ytrain)
rand_pred = rand_reg.predict(xtest)

# Comparing Model R2 score

Linear Regression Multivariable

Linear Regression Simple

Linear Regression Polynomial

Support Vector Regression

Decision Tree Regression

Random Forest Regression

# Results